# Polyhedral Geometry

* Author: Martin Bies, Luca Remke
* Version: OSCAR version 1.1.0 or newer.

This tutorial provides an introduction to polyhedral geometry in *OSCAR*.

The goal is to demonstrate basic constructions of and computations with convex polytopes, cones and polyhedral fans to newcomers to *OSCAR*. No prior knowledge of *OSCAR* is assumed.

The material discussed in this tutorial is selective. For a full description of the available functionality, please consult https://docs.oscar-system.org/stable/PolyhedralGeometry/intro/. Another very valuable resources is the article https://arxiv.org/abs/2308.07459, which provides an overview of polyhedral geometry in *OSCAR*.
 

## Content
1. [Polyhedra](#1.-Polyhedra)
   - [Polyhedra from *H*-representation](#1.A-Polyhedra-from-*H*-representation)
   - [Polyhedra from *V*-representation](#1.B-Polyhedra-from-*V*-representation)
   - [Polyhedra from existing Polyhedra](#1.C-Polyhedra-from-existing-Polyhedra)
   - [Properties and Attributes](#1.D-Properties-and-Attributes)
   - [Standard Constructions](#1.E-Standard-Constructions)
2. [Polyhedral Fans](#2.-Polyhedral-Fans)
   - [Polyhedral Cones](#2.A-Polyhedral-Cones)
   - [Polyhedral Fans](#2.B-Polyhedral-Fans)

In [1]:
using Oscar

  ___   ____   ____    _    ____
 / _ \ / ___| / ___|  / \  |  _ \   |  Combining ANTIC, GAP, Polymake, Singular
| | | |\___ \| |     / _ \ | |_) |  |  Type "?Oscar" for more information
| |_| | ___) | |___ / ___ \|  _ <   |  Manual: https://docs.oscar-system.org
 \___/ |____/ \____/_/   \_\_| \_\  |  1.1.0 #release-1.1 d0b75de 2024-06-21


# 1. Polyhedra

Let us begin by quoting the definition of polytopes and polyhedra from https://arxiv.org/pdf/2308.07459:

> A polyhedron in $\mathbb{R}^d$ is the intersection of finitely many affine closed half-spaces. Equivalently,
polyhedra can be described by finitely many linear inequalities, whence they are precisely the
feasible regions of linear programs. A polytope is a bounded polyhedron.

As such, it is not surprising that we can define polyhedra as the intersection of finitely many affine closed half-spaces. This is called an *H-representation*. Alternatively, it is possible to define polyhedra as the convex hull of finitely many points. The latter is referred to as *V-representation*. Both constructors are supported by *OSCAR* and we shall demonstrate them momentarily.

A third way of creating new polyhedra is to manipulate or "combine" existing polyhedra. For example, one can consider the so-called *Minkowski sum* of two (or finitely many) polyhedra. We will also demonstrate this.

Before we come to demonstrate those constructors, let us mention that *OSCAR* can draw pictures of polyhedra. This can often be very helpful to picture a geometry. The command for this is `visualize(P)`, where `P` is the polyhedron in question. Similarly, it is possible to save polyhedra to files and load them afterwards:
- `save("/path/to/your/desired/data/file", P)`,
- `load("/path/to/your/desired/data/file")`.

Note that he functionality of `visualize` as well as `save` and `load` extends also to cones and polyhedral fans, which we will discuss further down below.

## 1.A Polyhedra from *H*-representation

In *H-representation*, a polyhedron is given by $P = \{x \in \mathbb{R}^n \; \vert \; Ax \leq b\}$ for a matrix $A$ and a vector $b$. The following code creates the 2-simplex:

In [2]:
A = [-1 0; 0 -1; 1 1]

3×2 Matrix{Int64}:
 -1   0
  0  -1
  1   1

In [3]:
b = [0, 0, 1]

3-element Vector{Int64}:
 0
 0
 1

In [4]:
P1 = polyhedron(A, b)

Polyhedron in ambient dimension 2

Here is a somewhat compact/alternative code sample, which creates the same polyhedron:

In [5]:
I = ([-1 0; 0 -1; 1 1], [0, 0, 1])
P2 = polyhedron(I)

Polyhedron in ambient dimension 2

We can verify with *OSCAR* that those polyhedra are indeed identical:

In [6]:
P1 == P2

true

The *H* representation of a polyhedron can be recovered from the facets of *P1*:

In [7]:
facets(P1)

3-element SubObjectIterator{AffineHalfspace{QQFieldElem}} over the Halfspaces of R^2 described by:
-x_1 <= 0
-x_2 <= 0
x_1 + x_2 <= 1


Certainly, *OSCAR* can compute a large number of propertes and attributes of polyhedra and polytopes. The interested reader may consult the *OSCAR* documentation: https://docs.oscar-system.org/stable/PolyhedralGeometry/intro/. Here, we suffice it to demonstrate the computation of the volume of *P1*:

In [8]:
volume(P1)

1//2

## 1.B Polyhedra from *V*-representation

We now turn to the *V*-representation of polyhedra. In this case, we have $P = \mathrm{conv}(p_1, \ldots , p_N)$ where $p_i \in \mathbb{R}^n$. The points $p_i$ are vertices of the polyhdron in question.

While we will not elaborate much on the matter, let us mention that the *OSCAR* constructor also accepts rays and lineality spaces as optional input parameters. The interested reader may wish to consult the documentation (https://docs.oscar-system.org/stable/PolyhedralGeometry/intro/).

For an example, let us revisit the 2-simplex again. Its vertices $p_i$ are the origin $(0,0)$ as well as $(1,0)$ and $(0,1)$. Consequently, we construct the 2-simplex in *OSCAR* as follows:

In [9]:
V = [0 0; 1 0; 0 1]
P3 = convex_hull(V)

Polyhedron in ambient dimension 2

Let us verify that this gives us the same polyhedron as computed with the *H*-representations above:

In [10]:
P2 == P3

true

We can recover the information provided to the *V*-representation with the functions `vertices` (as well as `rays` and `lineality_space`):

In [11]:
vertices(P3)

3-element SubObjectIterator{PointVector{QQFieldElem}}:
 [0, 0]
 [1, 0]
 [0, 1]

Starting from a *V*-representation, the vertices are of course known upon construction. Not so about the facets, those need to be computed this time from the vertices:

In [12]:
facets(P3)

3-element SubObjectIterator{AffineHalfspace{QQFieldElem}} over the Halfspaces of R^2 described by:
-x_1 <= 0
-x_2 <= 0
x_1 + x_2 <= 1


## 1.C Polyhedra from existing Polyhedra

Polyhedra can be produced through various operations from other polyhedra. Here is a (most likely incomplete) list of available operations. More can be found in the documentation (https://docs.oscar-system.org/stable/PolyhedralGeometry/intro/):

| Method | Description |
| :----------- | :----------- |
| `+(P::Polyhedron, Q::Polyhedron)` | Minkowski sum $P+Q = \{x+y \; \vert \; x \in P, y \in Q\}$ von `P` und `Q`. |
| `*(k::Int, Q::Polyhedron)` | Scaled polyhedron  $kQ = \{kx \; \vert \; x \in Q\}$. |
| `*(P::Polyhedron, Q::Polyhedron)` | Cartesian product of `P` and `Q`. |
| `intersect(P::Polyhedron, Q::Polyhedron)` | Intersection $P \cap Q$ of `P` and `Q`. |
| `pyramid(P::Polyhedron, z::Number = 1)` | Pyramid over `P` with distance `z` between the vertex barycenter of `P` and the top of the pyramid. |
| `convex_hull(P::Polyhedron, Q::Polyhedron)` | Convex hull of `P` and `Q`. |

For a first example, let us compute the Minkowski sum of the 2-simplex $\Delta_2$ with itself:

In [13]:
D2 = simplex(2)

Polytope in ambient dimension 2

In [14]:
S1 = D2 + D2

Polyhedron in ambient dimension 2

In [15]:
vertices(S1)

3-element SubObjectIterator{PointVector{QQFieldElem}}:
 [0, 0]
 [2, 0]
 [0, 2]

So the Minkowski sum $\Delta_2 + \Delta_2$ of the standard 2-simplexes has the vertices $(0,0)$, $(2,0)$ and $(0,2)$. Actually, this is exactly the same polyhedron that we obtain by scaling $\Delta_2$ by $2$:

In [16]:
S2 = 2 * simplex(2)

Polytope in ambient dimension 2

In [17]:
S1 == S2

true

Finally, notice that $(\tfrac{1}{3}, \tfrac{1}{3})$ is the centroid of $\Delta_2$. Therefore, the pyramid of $\Delta_2$ has the vertex $(\tfrac{1}{3}, \tfrac{1}{3}, 1)$:

In [18]:
S3 = pyramid(simplex(2))

Polytope in ambient dimension 3

In [19]:
vertices(S3)

4-element SubObjectIterator{PointVector{QQFieldElem}}:
 [0, 0, 0]
 [1, 0, 0]
 [0, 1, 0]
 [1//3, 1//3, 1]

In [20]:
volume(S3)

1//6

In [21]:
facets(S3)

4-element SubObjectIterator{AffineHalfspace{QQFieldElem}} over the Halfspaces of R^3 described by:
-x_3 <= 0
-3*x_1 + x_3 <= 0
-3*x_2 + x_3 <= 0
3*x_1 + 3*x_2 + x_3 <= 3


## 1.D Properties and Attributes

We have already seen, that for a given polyhedron we can compute its vertices and facets. But there are many more properties and attributes that are supported within *OSCAR*. Let us provide some examples. To this end, we again focus on the 2-simplex $\Delta_2$.

We begin with `ambient_dim`, that is the integer $n$ such that - by construction - the simplex is embedded into $\mathbb{R}^n$. So for $Delta_2$, the ambient dimension is $2$:

In [22]:
ambient_dim(D2)

2

The simplex $\Delta_2$ itself has dimension $2$ and lies in codimension $0$ of $\mathbb{R}^2$:

In [23]:
dim(D2)

2

In [24]:
codim(D2)

0

This means that $\Delta_2$ is full dimensional:

In [25]:
is_fulldimensional(D2)

true

We can wonder if a polyhedron is bounded, i.e. is a polytope. Certainly, $\Delta_2$ is bounded:

In [26]:
is_bounded(D2)

true

For polytopes - by virtue of their boundedness - the number of lattice points contained in the polytope is finite. Therefore, we can execute the following function for $\Delta_2$:

In [27]:
lattice_points(D2)

3-element SubObjectIterator{PointVector{ZZRingElem}}:
 [0, 0]
 [0, 1]
 [1, 0]

Of course, we can also "just" count vertices or facets of polyhedra:

In [28]:
n_vertices(D2)

3

In [29]:
n_facets(D2)

3

There are many more properties and attributes for polyhedra supported by *OSCAR*. The following is a (likely incomplete) list. We again encourage interested readers to consult the documentation (https://docs.oscar-system.org/stable/PolyhedralGeometry/intro/):

| Method | Description |
| :----------- | :----------- |
| `facets(as::Type{T} = AffineHalfspace, P::Polyhedron)` | Facets of `P` in the format defined by `as` (`Halfspace` (Default), `Polyhedron` or `Pair`). |
| `vertices(P::Polyhedron)` | Iterator over the vertices of a polyhedron `P` as points. |
| `rays(P::Polyhedron)` | Minimal set of generators of the cone of unbounded directions of `P` (i.e. its rays) as points. |
| `rays_modulo_lineality(as, P::Polyhedron)` | Rays of the recession cone of `P` up to lineality as a `NamedTuple` with two iterators. |
| `minimal_faces(as, P::Polyhedron)` | Minimal faces of a polyhedron as a `NamedTuple` with two iterators. |
| `affine_hull(P::Polytope)` | (Affine) hyperplanes generating the affine hull of `P`.  |
| `ambient_dim(P::Polyhedron)` | Ambient dimension of `P`. |
| `dim(P::Polyhedron)` | Dimension of `P`. |
| `codim(P::Polyhedron)` | Codimension of `P`. |
| `is_bounded(P::Polyhedron)` | Checks whether `P` is bounded. |
| `is_feasible(P::Polyhedron)` | Checks whether `P` is non-empty. |
| `is_fulldimensional(P::Polyhedron)` | Checks whether `P` is full-dimensional. |
| `is_lattice_polytope(P::Polyhedron{QQFieldElem})` | Checks whether `P` is bounded and has integral vertices. |
| `lineality_dim(P::Polyhedron)` | Dimension of the largest affine subspace contained in `P`. |
| `lineality_space(P::Polyhedron)` | Matrix whose row span is the lineality space of `P`. |
| `recession_cone(P::Polyhedron)` | Recession cone of `P`. |
| `relative_interior_point(P::Polyhedron)` | A point in `P` not contained in any facet. |
| `n_facets(P::Polyhedron)` | Number of facets of `P`. |
| `n_vertices(P::Polyhedron)` | Number of vertices of `P`. |
| `f_vector(P::Polyhedron)` | Vector $(f_1, ..., f_{\dim(PF)-1})$, where $f_i$ is the number of faces of `P` of dimension $i$. |
| `g_vector(P::Polyhedron)` | Toric $g$-vector of a (bounded) polytope, defined by $g_0=1$ and $g_k = h_k - h_{k-1}$ for $1 \leq k \leq \lceil (d+1)/2 \rceil$, where $h$ is the $h$-vector and $d = \dim P$. |
| `h_vector(P::Polyhedron)` | Toric $h$-vector of a (bounded) polytope; for simplicial polytopes this is a linear transformation of the $f$-vector. |

## 1.E Standard Constructions

There are many famous standard constructions for polyhedra. A large number is supported by *OSCAR*. The supported constructions include the following. For a complete up-to-date list, please consult the documentation:

| Function | Description |
| :----------- | :----------- |
| `archimedean_solid(s::String)` | Archimedean solid with the name given by String `s`, see [here](https://docs.oscar-system.org/stable/PolyhedralGeometry/Polyhedra/constructions/). |
| `birkhoff(n::Integer, even::Bool = false)` | Birkhoff polytop of dimension `n`$^2$. |
| `catalan_solid(s::String)` | Catalan solid with the name given by String `s`, see [here](https://docs.oscar-system.org/stable/PolyhedralGeometry/Polyhedra/constructions/). |
| `cross_polytope([::Type{T} = QQFieldElem,] d::Int [,n::Rational])` | `d`-dimensional cross polytope around the origin with vertices located at $\pm e_i$ for each unit vector $e_i$ of $\mathbb{R}^d$, scaled by `n`. |
| `cube([::Type{T} = QQFieldElem,] d::Int , [l::Rational = -1, u::Rational = 1])` | $[l,u]$-cube in dimension `d`. |
| `cyclic_polytope(d::Int, n::Int)` | Cyclic polytope that is the convex hull of `n` points on the moment curve in dimension `d`. |
| `del_pezzo_polytope(d::Int)` | `d`-dimensional del Pezzo polytop. |
| `dodecahedron()` | Regular dodecahedron, one of the Platonic solids. |
| `fano_simplex(d::Int)` | A lattice simplex such that the origin is the unique interior lattice point. |
| `fractional_cut_polytope(G::Graph{Undirected})` | The fractional cut polytope of the graph `G`. |
| `fractional_matching_polytope(G::Graph{Undirected})` | The fractional matching polytope of the graph `G`. |
| `gelfand_tsetlin(lambda::AbstractVector)` | Gelfand Tsetlin polytope indexed by a weakly decreasing vector `lambda`. |
| `icosahedron()` | Regular icosahedron, one of the Platonic solids. |
| `johnson_solid(i::Int)` | `i`-th proper Johnson solid. |
| `newton_polytope(poly::Polynomial)` | Newton polytope of the multivariate polynomial `poly`. |
| `orbit_polytope(V::AbstractCollection[PointVector], G::PermGroup)` | Convex hull of the orbit of one or several points (given row-wise in `V`) under the action of `G`. |
| `platonic_solid(s::String)` | Platonic solid with the name given by String `s`, see [here](https://docs.oscar-system.org/stable/PolyhedralGeometry/Polyhedra/constructions/). |
| `rand_spherical_polytope([rng::AbstractRNG,] d::Int, n::Int;` <br> `distribution=:uniform, precision=nothing, seed=nothing)` | The convex hull of `n` points on the unit sphere in $\mathbb{R}^d$. |
| `regular_24_cell()` | Regular 24-cell, one out of three exceptional regular 4-polytopes. |
| `regular_120_cell()` | Regular 120-cell, one out of three exceptional regular 4-polytopes. |
| `regular_600_cell()` | Regular 600-cell, one out of three exceptional regular 4-polytopes. |
| `simplex([::Type{T} = QQFieldElem,] d::Int [,n::Rational])` | Simplex which is the convex hull of the standard basis vectors along with the origin in $\mathbb{R}^d$, scaled by `n`. |
| `tetrahedron()` | Regular tetrahedron, one of the Platonic solids. |


# 2. Polyhedral Fans

There are multiple ways to motivate polyhedral fans. With a view towards algebraic geometry, one notices that polyhedral fans are the (polyhedral) backbone of toric varieties. Toric varities in turn are an arguably vast class of algebraic varieties, for which it is algorithmically possible to determine a wide range of properties and attributes (e.g. the Chow ring). As such toric varieties are of ample importance to (constructive) algebraic geometry. While polyhedral fans certainly have more applications than just toric geometry, we hope that this is sufficient to emphasize their importance.

A polyhedral fan is a collection of polyhedral cones. In staying with the picture of toric varieties, we may think of the polyhedral cones as affine charts, that are glued together in a consistent fashion. Consequently, a collection of polyhedral cones $F$ forms a polyhedral fan if the following two properties are satisfied:
1. Each face of each cone in $F$ is, itself, an element of $F$.
2. For any two cones $C_1$, $C_2$ in $F$, their intersection $C_1 \cap C_2$ is also an element of $F$.

Polyhedral cones in turn are special instances of polyhedra. We first demonstrate the creation and available functionality for polyhedral cones before we come back to constructing polyhedral fans.

## 2.A Polyhedral Cones

We construct a polyhedral cone by specifying its ray generators and then computing the positive hull of these ray generators. Here is an example:

In [30]:
R = [2 -1; 0 1]
C = positive_hull(R)

Polyhedral cone in ambient dimension 2

Design choices of *OSCAR* lead to half-integer ray vectors:

In [31]:
rays(C)

2-element SubObjectIterator{RayVector{QQFieldElem}}:
 [1, -1//2]
 [0, 1]

We can force integer values for the rays as follows:

In [32]:
matrix(ZZ, rays(C))

[2   -1]
[0    1]

Since polyhedral cones are special instances of polyhedra, the polyhedra functionality applies. For instance we can ask for the dimension, ambient dimension and codimension:

In [33]:
dim(C)

2

In [34]:
ambient_dim(C)

2

In [35]:
codim(C)

0

However, there are also new functions. For instance, we can check if a cone is pointed (that is, if there is no lineality space of positive dimension in C):

In [36]:
is_pointed(C)

true

We can also compute the polar dual cone:

In [37]:
pdC = polarize(C)

Polyhedral cone in ambient dimension 2

In [38]:
rays(pdC)

2-element SubObjectIterator{RayVector{QQFieldElem}}:
 [1, 0]
 [1, 2]

We can also compute a Hilbert basis of the lattice points of a cone:

In [39]:
hilbert_basis(C)

3-element SubObjectIterator{PointVector{ZZRingElem}}:
 [1, 0]
 [0, 1]
 [2, -1]

The informed reader may already have noticed it from the definition, or noticed it from the computation of the Hilbert basis: The cone *C* is not smooth. Even that we can test:

In [40]:
is_smooth(C)

false

We conclude with a (most definitely incomplete) list of available functionality for cones. The interested reader should consult the documentation (https://docs.oscar-system.org/stable/PolyhedralGeometry/intro/):

| Method | Description |
| :----------- | :----------- |
| `ambient_dim(C::Cone)` | Ambient dimension of  `C`. |
| `in(v::AbstractVector, C::Cone)` | Checks whether the vector `v` is contained in the cone `C`. |
| `issubset(C0::Cone, C1::Cone)` | Checks whether `C0` is a subset of the cone `C1`. |
| `f_vector(C::Cone)` | Vector $(f_1, ..., f_{dim(PF)-1})$, where $f_i$ is the number of faces of `C` of dimension $i$. |
| `hilbert_basis(C::Cone{QQFieldElem})` | Hilbert basis of a pointed cone `C` as the rows of a matrix. |
| `codim(C::Cone)` | Codimension of `C`. |
| `dim(C::Cone)` | Dimension of `C`. |
| `polarize(C::Cone)` | Polar dual of `P`. |
| `intersect(C0::Cone{T}, C1::Cone{T}) where T<:scalar_types` | Intersection $C0 \cap C1$ of `C0` and `C1`. |
| `is_pointed(C::Cone)` | Determines whether `C` is pointed. |
| `is_fulldimensional(C::Cone)` | Determines whether `C` is full-dimensional. |
| `lineality_dim(C::Cone)` | Dimension of the largest linear subspace contained in `C`. |
| `lineality_space(C::Cone)` | Basis of the lineality space of `C`. |
| `n_facets(C::Cone)` | Number of facets of a cone `C`. |
| `n_rays(C::Cone)` | Number of rays of `C`. |
| `rays(C::Cone)` | Rays of `C`. |
| `rays_modulo_lineality(as, C::Cone)` | Rays of the cone of `C` up to lineality as a `NamedTuple` with two iterators.  |

## 2.B Polyhedral Fans

To construct a polyhedral fan, we must provide a list of polyhedral cones. While possible, there is a more efficient way to construct polyhedral fans. First, we do not need the information of all cones, but only the maximal cones. Those are the cones of highest dimension. All other cones in the polyhedral fan are obtained from intersections of these maximal cones. With that being said, we typically collect the rays of all maximal cones into a list and then provide a second argument that states which rays generate maximal cones:
- `Rays`: The rays of the fan.
- `MC_reps`: IncidenceMatrix whose rows give the indices of the rays which generate maximal cones.

For an example, consider the fan $\Sigma_{\mathbb{P}^2}$ associated with the projective plane $\mathbb{P}^2$. This fan has ray generators $e_1$, $e_2$ and $-e_1-e_2$ and the maximal cones are:
* $C_1 = \mathrm{Cone}(e_1, e_2)$,
* $C_2 = \mathrm{Cone}(e_1, - e_1 - e_2)$,
* $C_3 = \mathrm{Cone}(e_2, - e_1 - e_2)$.
In *OSCAR*, we construct the polyhedral fan as follows:

In [41]:
R = [1 0; 0 1; -1 -1]
IM = IncidenceMatrix([[1, 2], [2, 3], [1, 3]])
PF = polyhedral_fan(IM, R)

Polyhedral fan in ambient dimension 2

Just as for cones and polyhedra, we can ask about the (ambient) dimension of polyhedral fans:

In [42]:
ambient_dim(PF)

2

In [43]:
dim(PF)

2

The so-called `f_vector` tells us how many cones of dimensions 1, 2, 3, ... do exist in a polyhedral fan. For *PF*, there are three 1-dimensional and three 2-dimensional cones and no higher dimensional cones. So the answer is (3,3):

In [44]:
f_vector(PF)

2-element Vector{ZZRingElem}:
 3
 3

A polyhedral fan is pointed, smooth, simplicial, etc. if all of its polyhedral cones are so:

In [45]:
is_pointed(PF)

true

In [46]:
is_smooth(PF)

true

In [47]:
is_simplicial(PF)

true

There is also the notion of completeness, which means that the union of all cones in the polyhedral fan cover all of the ambient space. For *PF*, this ambient space is $\mathbb{R}^2$. Indeed, $C_1 \cup C_2 \cup C_3 = \mathbb{R}^2$, and so *PF* is complete. (This is parallel to the projective space $\mathbb{P}^2$ being complete):

In [48]:
is_complete(PF)

true

Many more properties and attributes for polyhedral fans are supported. Here is a (most likely incomplete) list. The interested reader may consult the documentation (https://docs.oscar-system.org/stable/PolyhedralGeometry/intro/):

| Methode | Rückgabe |
| :----------- | :----------- |
| `ambient_dim(PF::PolyhedralFan)` | Ambient dimension of `PF`. |
| `dim(PF::PolyhedralFan)` | Dimension of `PF`. |
| `f_vector(PF::PolyhedralFan)` | Vector $(f_1, ..., f_{dim(PF)-1})$, where $f_i$ is the number of faces of `PF` of dimension $i$. |
| `is_complete(PF::PolyhedralFan)` | Determines whether `PF` is complete. |
| `is_pointed(PF::PolyhedralFan)` | Determines whether `PF` is pointed. |
| `is_regular(PF::PolyhedralFan)` | Determines whether `PF` is regular. |
| `is_simplicial(PF::PolyhedralFan)` | Determines whether `PF` is simpicial. |
| `is_smooth(PF::PolyhedralFan{fmpq})` | Determines whether `PF` is smooth. |
| `lineality_dim(PF::PolyhedralFan)` | Dimension of the largest linear subspace of `PF`. |
| `lineality_space(PF::PolyhedralFan)` | Non-redundant matrix whose rows are generators of the lineality space of  `PF`. |
| `maximal_cones(PF::PolyhedralFan)` | Maximal cones of  `PF`. |
| `cones(PF::PolyhedralFan, cone_dim::Int)` | Iterator over the cones of `PF` of dimension `cone_dim`. |
| `cones(PF::PolyhedralFan)` | Ray indices of all non-zero-dimensional cones in a polyhedral fan `PF`. |
| `n_maximal_cones(PF::PolyhedralFan)` | Number of maximal cones of `PF`. |
| `n_cones(PF::PolyhedralFan)` | Number of cones of `PF`. |
| `n_rays(PF::PolyhedralFan)` | Number of rays of `PF`. |
| `rays(PF::PolyhedralFan)` | Rays of `P`. |
| `rays_modulo_lineality(as, F::PolyhedralFan)` | Rays of the polyhedral fan `F` up to lineality as a `NamedTuple` with two iterators. |
| `primitive_collections(PF::PolyhedralFan)` | Primitive collections of `PF`. |
| `starsubdivision(PF::PolyhedralFan, n::Int)` | Star subdivision of a polyhedral fan `PF` at its `n`-th maximal torus orbit. |
| `*(PF1::PolyhedralFan, PF2::PolyhedralFan)` | Cartesian/direct product of two polyhedral fans `PF1` and `PF2`. |